# Introduction

In a [previous post](//ccom.uprrp.edu/~humberto/very-small-rsa-example.html) we did a very small example of RSA, following [the tutorial by Barry Steyn](http://doctrina.org/How-RSA-Works-With-Examples.html). For working with modular integer arithmetic we used functions from a [toy RSA implementation by Alex Roper](https://github.com/calmofthestorm/toys/blob/master/rsa/rsa.py).

That toy implementation has functions for doing plain old RSA with larger keys, and support for encoding strings (and files, from the command line). We're going to use them today to encrypt and sign simple messages.

# Generating keys

With the rsa module, we can generate keys of a specified number of bits. Keys smaller than 2048 bits are not considered secure, but for this example we'll use smaller keys to speed up key generation.

In [1]:
import rsa

foo = rsa.RSAPrivateKey(128)

Even with the reduced key size we specified, the public modulus is a very large number. Factoring this would require serious resources.

In [2]:
foo.N

55622303509980058401201317471604932276540414230050725790181357844968239059357L

As before, the public key is (e, N) and the private key is (d, N).

In [3]:
foo.e

5

In [4]:
foo.d

44497842807984046720961053977283945820848353659736187103254386998593893054845L

I modified rsa.py to store p, q, and phi(n), so I can dig around the internals of RSA.

In [5]:
foo.phi

55622303509980058401201317471604932276060442074670233879067983748242366318556L

We can quickly verify that e and d are multiplicative inverses in the integers modulo phi(n).

In [6]:
(foo.e * foo.d) % foo.phi

1L

# Encryption

If foo wants to send an encrypted message to bar, he will need bar's public key.

In [7]:
bar = rsa.RSAPrivateKey(128)
barpub = bar.GetPublicKey()

We will prepare a message to send to bar.

In [8]:
m = "Attack at dawn. This is a really long message to see if I can get rsa \
to encode it as more than one number."

Recall from last post that we can only encode numbers smaller than n. If we wish to send a longer message, we need to divide it into smaller pieces and send a sequence. For this we need to know bar's public modulus.

In [9]:
encoded = rsa.Message.Encode(m, barpub.N)
encoded.numbers

[115648799396826751567766945599864979770100194381312838842529118569426414700L,
 214012274130603322894259128424238691739369727293888509978752859858830192160L,
 182685454824088443191059084189136997345550917941544886668325422864334353512L,
 172144231835021845661017141195722362689495106685351054972168510011550466048L]

Now we are ready to compute the cyphertext. This transforms the encoded message into a cyphertext that only bar's private key can decrypt.

In [10]:
c = barpub.Encrypt(encoded)

In [11]:
c.numbers

[69689972244802969832083520729118788876390426116318231038863288408383716065964L,
 23567992220645916473090615843132509658144614908574188924254236793280978342508L,
 88415518404081156810369998156677379257013186995307027556270425798465542648L,
 53111282119334505415721630236019698285589047904225912165489505527984338285981L]

In [12]:
bar.Decrypt(c).Decode()

'Attack at dawn. This is a really long message to see if I can get rsa to encode it as more than one number.'

# Signatures

A digital signature is just a message that has been encrypted with someone's private key. Thus anyone with the public key can verify that the message comes from the owner of the private key, and that no one has altered the message. 

In [13]:
reply = "This is really bar."
encoded_reply = rsa.Message.Encode(reply, bar.N)
encoded_reply.numbers

[149135777980097582635656205376687025111355119538120755623372731861778300928L]

We have to compute the signature manually, as the rsa module won't let us do it directly.

In [14]:
s = rsa.modexp(encoded_reply.numbers[0], bar.d, bar.N)

In [15]:
s

79482082372224077031364631212447551920995692144876852609977919004935413771073L

If foo receives a message claiming to come from bar, he can use bar's public key to confirm.

In [16]:
verify = encoded_reply
verify.numbers = [rsa.modexp(s, barpub.e, barpub.N)]

In [17]:
plaintext = rsa.Message.Decode(verify)
plaintext

'This is really bar.'

In [18]:
plaintext == reply

True